In [1]:
import os
# from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_ollama import ChatOllama
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, Tool
from dotenv import load_dotenv

load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGSMITH_API"] = os.getenv("LANGSMITH_API")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_PROJECT"] = "Explore Evaluating index using LLM" 

from langchain.agents import Tool 


llm = ChatOllama(
    model = "llama3.1",
    base_url = "http://localhost:11434")




def get_function_tools():
  search = TavilySearchAPIWrapper()
  tavily_tool = TavilySearchResults(api_wrapper=search)

  tools = [
      tavily_tool
  ]

  tools.extend(load_tools(['wikipedia']))

  return tools

In [ ]:
prompt = hub.pull("hwchase17/react")
tools = get_function_tools()
agent = create_react_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

question = """Who is the greatest astronaut?"""
question = """Roger has 3 tennis balls. He bought 2 more cans of tennis balls. Each can ontains 5 tennis balls. How many tennis balls does Roger has?"""
question = """What is the age difference between Amitabh Bachchan and Abhishek Bachchan?"""

output = agent_executor.invoke({"input": question})
process_chunks(output)

/Users/suranjanasamanta/opt/anaconda3/envs/promptEngineering/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(




> Entering new AgentExecutor chain...
Thought: To find the age difference, I need to know the birth dates of Amitabh Bachchan and Abhishek Bachchan.
Action: wikipedia
Action Input: search query for Amitabh Bachchan's birth datePage: List of films considered the worst
Summary: The films listed below have been cited by a variety of notable critics in varying media sources as being among the worst films ever made. Examples of such sources include Metacritic, Roger Ebert's list of most-hated films, The Golden Turkey Awards, Leonard Maltin's Movie Guide, Rotten Tomatoes, pop culture writer Nathan Rabin's My World of Flops, the Stinkers Bad Movie Awards, the cult TV series Mystery Science Theater 3000 (alongside spinoffs Cinematic Titanic, The Film Crew and RiffTrax), and the Golden Raspberry Awards (aka the "Razzies"). Films on these lists are generally feature-length films that are commercial/artistic in nature (intended to turn a profit, express personal statements or both), professiona

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from rich.console import Console

# Initialize dotenv to load environment variables
load_dotenv()

# Initialize Rich for better output formatting and visualization
rich = Console()

# Initialize OpenAI LLM

# Initialize Tavily
tavily = TavilySearchResults(max_results=3)

# Create a LangGraph agent
langgraph_agent = create_react_agent(model=llm, tools=[tavily])


# Define a function to process chunks from the agent
def process_chunks(chunk):
    """
    Process a chunk of data and extract information about tool calls made by the agent.

    Parameters:
        chunk (dict): A dictionary containing information about the agent's messages.

    Returns:
        None

    This function processes a chunk of data and checks if it contains information about an agent. If the chunk contains
    an agent's message, it iterates over the messages in the agent's messages. For each message, it checks if the
    message contains tool calls. If a tool call is found, the function extracts the tool name and query from the
    message and prints a formatted message using the Rich library. If no tool call is found, the function extracts
    the agent's answer from the message and prints it using the Rich library.

    The function uses the Rich library for formatting and printing the messages.
    """

    # Check if the chunk contains an agent's message
    if "agent" in chunk:
        # Iterate over the messages in the chunk
        for message in chunk["agent"]["messages"]:
            # Check if the message contains tool calls
            if "tool_calls" in message.additional_kwargs:
                # If the message contains tool calls, extract and display an informative message with tool call details

                # Extract all the tool calls
                tool_calls = message.additional_kwargs["tool_calls"]

                # Iterate over the tool calls
                for tool_call in tool_calls:
                    # Extract the tool name
                    tool_name = tool_call["function"]["name"]

                    # Extract the tool query
                    tool_arguments = eval(tool_call["function"]["arguments"])
                    tool_query = tool_arguments["query"]

                    # Display an informative message with tool call details
                    rich.print(
                        f"\nThe agent is calling the tool [on deep_sky_blue1]{tool_name}[/on deep_sky_blue1] with the query [on deep_sky_blue1]{tool_query}[/on deep_sky_blue1]. Please wait for the agent's answer[deep_sky_blue1]...[/deep_sky_blue1]",
                        style="deep_sky_blue1",
                    )
            else:
                # If the message doesn't contain tool calls, extract and display the agent's answer

                # Extract the agent's answer
                agent_anser = message.content

                # Display the agent's answer
                rich.print(f"\nAgent:\n{agent_anser}", style="black on white")


# Loop until the user chooses to quit the chat
while True:
    # Get the user's question and display it in the terminal
    user_question = input("\nUser:\n")

    # Check if the user wants to quit the chat
    if user_question.lower() == "quit":
        rich.print("\nAgent:\nHave a nice day! :wave:\n", style="black on white")
        break

    # Use the stream method of the LangGraph agent to get the agent's answer
    for chunk in langgraph_agent.stream(
        {"messages": [HumanMessage(content=user_question)]}
    ):
        # Process the chunks from the agent
        process_chunks(chunk)
